In [2]:
pip install pulp


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pulp
import pandas as pd
import numpy as np
from math import sqrt

# Define cities and data
cities = [
    {"City": 1, "X": 0, "Y": 0, "Visits": 9000},
    {"City": 2, "X": 16, "Y": 9, "Visits": 6000},
    {"City": 3, "X": 18, "Y": 6, "Visits": 8000},
    {"City": 4, "X": 10, "Y": 18, "Visits": 7000},
    {"City": 5, "X": 8, "Y": 10, "Visits": 6000},
    {"City": 6, "X": 14, "Y": 16, "Visits": 1000},
    {"City": 7, "X": 12, "Y": 3, "Visits": 3000},
    {"City": 8, "X": 12, "Y": 15, "Visits": 2000},
    {"City": 9, "X": 14, "Y": 13, "Visits": 3000}
]

# Calculate distance matrix
num_cities = len(cities)
distance_matrix = np.zeros((num_cities, num_cities))
for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            distance_matrix[i, j] = sqrt((cities[i]["X"] - cities[j]["X"])**2 + (cities[i]["Y"] - cities[j]["Y"])**2)

# Set up the LP problem
prob = pulp.LpProblem("Hospital_Location_Optimization", pulp.LpMinimize)

# Define decision variables
X = [pulp.LpVariable(f"X_{i}", cat="Binary") for i in range(num_cities)]  # 1 if hospital is built in city i
Y = [[pulp.LpVariable(f"Y_{i}_{j}", cat="Binary") for j in range(num_cities)] for i in range(num_cities)]  # 1 if city j is assigned to hospital in city i

# Define the objective function
prob += pulp.lpSum(cities[j]["Visits"] * distance_matrix[i][j] * Y[i][j] for i in range(num_cities) for j in range(num_cities)), "Total_Travel_Distance"

# Constraint: Only two hospitals can be built
prob += pulp.lpSum(X[i] for i in range(num_cities)) == 2, "Two_Hospitals_Constraint"

# Constraint: Each city must be assigned to one hospital
for j in range(num_cities):
    prob += pulp.lpSum(Y[i][j] for i in range(num_cities)) == 1, f"Assignment_Constraint_for_City_{j}"

# Constraint: A city can only be assigned to a hospital that exists
for i in range(num_cities):
    for j in range(num_cities):
        prob += Y[i][j] <= X[i], f"Assignment_to_Hospital_Constraint_{i}_{j}"

# Solve the problem
prob.solve()

# Print the results
print("Status:", pulp.LpStatus[prob.status])
for i in range(num_cities):
    print(f"City {i+1}: Hospital = {pulp.value(X[i])}")
    for j in range(num_cities):
        if pulp.value(Y[i][j]) == 1:
            print(f" - City {j+1} is assigned to hospital in City {i+1}")
print("Total Travel Distance:", pulp.value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_r/6ppttwts2vn32pmq4vn9qt6w0000gn/T/f9a26c54b619416780924dd15896d034-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_r/6ppttwts2vn32pmq4vn9qt6w0000gn/T/f9a26c54b619416780924dd15896d034-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 96 COLUMNS
At line 601 RHS
At line 693 BOUNDS
At line 784 ENDATA
Problem MODEL has 91 rows, 90 columns and 252 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 209687 - 0.00 seconds
Cgl0004I processed model has 91 rows, 90 columns (90 integer (90 of which binary)) and 252 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 209687
Cbc0038I Before mini branch and bound, 90 integers at bound fixed and 0 contin